<a href="https://colab.research.google.com/github/AyushMaria/Vortex-Detection/blob/main/Core_detection_(Voting_Classifier).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive

In [2]:
# Mount the Google Drive at mount
mount='/content/gdrive'
drive.mount(mount, force_remount=True)
# Switch to the directory on the Google Drive that you want to use
drive_root=mount + '/My Drive/Colab Notebooks/Object Detection'

Mounted at /content/gdrive


In [3]:
%cd $drive_root
%pwd

/content/gdrive/My Drive/Colab Notebooks/Object Detection


'/content/gdrive/My Drive/Colab Notebooks/Object Detection'

In [4]:
data = pd.read_csv('adaboost.csv')
data.head()

,ID,contourArea,areaPercDiff,aspectRatio,momentLocDiff,label
0,1,283.5,67.500000,0.952381,28.491470,vortex
1,2,2281.5,52.388060,1.030769,24.713409,vortex
2,3,1177.0,53.891941,1.238095,24.540352,vortex
3,4,2285.0,32.503556,0.778947,51.181107,no_vortex
4,5,3155.5,63.490946,1.014286,51.263397,vortex


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             36 non-null     int64  
 1   contourArea    36 non-null     float64
 2   areaPercDiff   36 non-null     float64
 3   aspectRatio    36 non-null     float64
 4   momentLocDiff  36 non-null     float64
 5   label          36 non-null     object 
dtypes: float64(4), int64(1), object(1)
memory usage: 1.8+ KB


In [46]:
X = data[['contourArea','areaPercDiff','aspectRatio','momentLocDiff']]
X.head()

,contourArea,areaPercDiff,aspectRatio,momentLocDiff
0,283.5,67.500000,0.952381,28.491470
1,2281.5,52.388060,1.030769,24.713409
2,1177.0,53.891941,1.238095,24.540352
3,2285.0,32.503556,0.778947,51.181107
4,3155.5,63.490946,1.014286,51.263397


In [47]:
print(X.iloc[:,:].min()-1)

contourArea      208.000000
areaPercDiff      19.064935
aspectRatio       -0.587786
momentLocDiff      1.230667
dtype: float64


In [49]:
y = data['label']
y.head()

0       vortex
1       vortex
2       vortex
3    no_vortex
4       vortex
Name: label, dtype: object

In [77]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
#y=le.fit_transform(y)
y.head()

0       vortex
1       vortex
2       vortex
3    no_vortex
4       vortex
Name: label, dtype: object

In [78]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [79]:
# Import the AdaBoost classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from itertools import product


# Create adaboost classifer object
clf1 = AdaBoostClassifier(n_estimators=100, learning_rate=1, random_state=1)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=1)
clf3 = XGBClassifier(n_estimators=100, learning_rate=1, random_state=1)


# Train Adaboost Classifer
model1 = clf1.fit(X_train, y_train)
model2 = clf2.fit(X_train, y_train)
model3 = clf3.fit(X_train, y_train)

#Predict the response for test dataset
eclf = VotingClassifier( estimators=[('abc', clf1), ('rf', clf2), ('xgb', clf3)],voting='hard')
model4 = eclf.fit(X_train, y_train)

for clf, label in zip([clf1, clf2, clf3, eclf], ['AdaBoost Classifier', 'Random Forest', 'XGBoost Classifier', 'Ensemble']):
  scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))



Accuracy: 0.76 (+/- 0.17) [AdaBoost Classifier]
Accuracy: 0.73 (+/- 0.07) [Random Forest]
Accuracy: 0.83 (+/- 0.11) [XGBoost Classifier]
Accuracy: 0.78 (+/- 0.13) [Ensemble]


In [80]:
y_pred = eclf.predict(X_test)

In [81]:
n=[]
for row in X_test.itertuples():
  n.append(row[0]+1)
  
for i in range(8):
  print(n[i], y_pred[i])

15 vortex
19 vortex
27 no_vortex
22 no_vortex
31 vortex
34 vortex
21 vortex
3 vortex


In [82]:
eclf = VotingClassifier( estimators=[('abc', clf1), ('rf', clf2), ('xgb', clf3)],voting='soft',weights=[1,1,1])
model4 = eclf.fit(X_train, y_train)

for clf, label in zip([clf1, clf2, clf3, eclf], ['AdaBoost Classifier', 'Random Forest', 'XGBoost Classifier', 'Ensemble']):
  scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


Accuracy: 0.76 (+/- 0.17) [AdaBoost Classifier]
Accuracy: 0.73 (+/- 0.07) [Random Forest]
Accuracy: 0.83 (+/- 0.11) [XGBoost Classifier]
Accuracy: 0.75 (+/- 0.13) [Ensemble]


In [83]:
y_pred = eclf.predict(X_test)

In [84]:
n=[]
for row in X_test.itertuples():
  n.append(row[0]+1)
  
for i in range(8):
  print(n[i], y_pred[i])

15 vortex
19 vortex
27 no_vortex
22 no_vortex
31 vortex
34 vortex
21 vortex
3 vortex
